### Laboratorium 9 - zadania

Rozpakuj dołączone archiwum `dataset.zip` w folderze z notebookiem przed rozpoczęciem pracy.  
Potrzebne Ci będą pakiety `scikit-learn` i `scikit-image` - doinstaluj, jeśli jeszcze nie posiadasz.

Na tym laboratorium przetestujesz uproszczoną wersję detektora Violi-Jonesa dla obrazów twarzy. Uproszczenie polegało będzie na tym, że nie będziemy stosować pełnego pipeline'u kaskadowego, gdyż jego nauczenie jest dość skomplikowanym i długotrwałym procesem. Zastosowanie takiego gotowego detektora jest banalnie proste, dlatego internet roi się od lepszych i gorszych tutoriali. Dobre przykłady znajdziesz w dokumentacji [scikit-image](https://scikit-image.org/docs/dev/auto_examples/applications/plot_face_detection.html) oraz [OpenCV](https://docs.opencv.org/4.5.4/db/d28/tutorial_cascade_classifier.html) - na obu stronach znajdziesz także pliki konfiguracyjne dla nauczonych detektorów.

Funkcje (linki do dokumentacji), które należy mieć pod ręką na tym laboratorium:
* [`skimage.transform.integral_image`](https://scikit-image.org/docs/dev/api/skimage.transform.html?highlight=integral_image#skimage.transform.integral_image)
* [`skimage.feature.haar_like_feature`](https://scikit-image.org/docs/stable/api/skimage.feature.html#haar-like-feature) - oblicza (opcjonalnie: wybrane) cechy Haara dla podanego obrazu całkowego
* [`skimage.feature.haar_like_feature_coord`](https://scikit-image.org/docs/stable/api/skimage.feature.html#haar-like-feature-coord) - wylicza koordynaty cech Haara
* [`skimage.feature.draw_haar_like_feature`](https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.draw_haar_like_feature) - rysuje cechy Haara o podanych koordynatach
* [`sklearn.ensemble.AdaBoostClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

Przykładów użycia nie zamieszczam w instrukcji bo są świetne w dokumentacji.

In [9]:
import os
import time

import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import integral_image
from skimage.feature import haar_like_feature
from sklearn.ensemble import AdaBoostClassifier
import sklearn.tree

Poniższa komórka wczyta cały dataset w postaci pary (lista obrazów, lista etykiet). Obrazy skonwertuje od razu do skali szarości, ale skalowanie musisz wykonać własnoręcznie.

In [6]:
def read_folder(path):
    return list(cv2.imread(item.path, flags=cv2.IMREAD_GRAYSCALE) for item in os.scandir(path))

positive_images = read_folder('natural_images_postprocessed/face')
negative_images = read_folder('natural_images_postprocessed/noface')

raw_images = positive_images + negative_images
raw_targets = np.asarray(list(1 for _ in positive_images ) + list(0 for _ in negative_images), dtype=np.int64)

Poniższa komórka ładuje obrazy testowe do części badawczej.

In [ ]:
test_images = read_folder('test_images')

---

#### Zadanie 1 (0.5 pkt | 0.0 pkt)

Celem tego zadania jest nauczenie się korzystania z `haar_like_feature` i `haar_like_feature_coord`.

##### Zadanie 1a

Przetestuj pipeline do ekstrakcji cech Haara dla pojedynczego obrazu. Pamiętaj o skalowaniu! Sprawdź, ile cech otrzymujesz dla różnych skal pomiędzy 16x16 a 32x32. Zwróć uwagę, że `haar_like_feature` domyślnie ekstrahuje wszystkie możliwe cechy - być może wystarczą tylko niektóre z nich (np. tylko `type-2-x` i `type-2-y`).

##### Zadanie 1b 

Użyj `haar_like_feature_coord` do wyznaczenia koordynat cech Haara dla podanej konfiguracji (rozmiar obrazka, wybrane typy cech). Wartości zwracane przez tę funkcję to para (coords, types) w nieco dziwnym typie:
* coords to `np.array` o długości N=liczba cech, zawierający pythonowe listy dwuelementowe, zawierające punkty początkowy i końcowy prostokąta obejmującego daną cechę,
* types to `np.array` o długości N=liczba cech, zawierający stringi identyfikujące typ danej cechy.

Wybierz jedną dowolną cechę (na chybił trafił) i wyświetl ją używając `draw_haar_like_feature`.

---

#### Zadanie 2 (0.75 pkt | 0.25 pkt)

Dokonaj ekstrakcji cech Haara z całego datasetu. Dobierz parametry (skalowanie, typy cech) w taki sposób, by ekstrakcja nie trwała więcej niż 2-5 minut. Uformuj zbiory uczący i walidacyjny na potrzeby uczenia klasyfikatora, pamiętając o losowym samplowaniu z obu klas (np. używając [`sklearn.model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)).
Wykonaj uczenie klasyfikatora AdaBoost na tym zbiorze, zaczynając od niewielkiej liczby korzeni (N=10), powtarzając 2-3 razy dla większych wartości - ten proces również nie powinien trwać dłużej niż minutę. Na każdym etapie sprawdź, które cechy zostały wybrane.

In [7]:
classifier = AdaBoostClassifier(n_estimators=3)

Poszczególne składniki klasyfikatora znajdują się w atrybucie `estimators_` (tworzonym dopiero po sfitowaniu). Aby dobrać się do konkretnych drzew (korzeni) decyzyjnych:

In [ ]:
tree_id = 0
my_tree = classifier.estimators_[tree_id].tree_ # konkretne drzewo (korzeń) -- NIE ZADZIAŁA PRZED NAUCZENIEM
print(my_tree.feature)   # cecha skojarzona z korzeniem; zwróć uwagę, że tylko pierwszy element zawiera id konkretnej cechy
print(my_tree.threshold) # próg decyzyjny dla tej cechy
print(sklearn.tree.export_text(classifier.estimators_[tree_id])) # pretty-printing powyższego, obiekt musi mieć atrybut 'tree_'

---

#### Zadanie 3 (1.0 pkt | 0.25 pkt)

Dla wybranego obrazu testowego wykonaj inferencję wykorzystując podejście *sliding-sindow*:
* przeskaluj obraz źródłowy (możesz założyć a priori, że wiadomo jakiego rozmiaru będą twarze),
* oblicz obraz całkowy,
* dla każdego RoI wykonaj ekstrakcję cech Haara...
* ...a następnie ich klasyfikację;
* jeśli jest pozytywna - zapisz koordynaty danego RoI.

Do prezentacji wyników możesz wykorzystać [`cv2.rectangle`](https://docs.opencv.org/4.5.4/d6/d6e/group__imgproc__draw.html#ga07d2f74cadcf8e305e810ce8eed13bc9) (przeskaluj wymiary znalezionego RoI w górę, by narysować bounding-boksa na obrazie oryginalnym).

---

#### Zadanie 4 (0.75 pkt | 0.0 pkt)

Dokonaj optymalizacji algorytmu detekcji cech.

##### Zadanie 4a (0.25 pkt)

Ze znalezionych korzeni decyzyjnych łatwo wyciągniesz indeksy cech znaczących - wykorzystaj je oraz funkcję `haar_like_feature_coord` do znalezienia podzbioru koordynat cech Haara, które następnie przekażesz do funkcji `haar_like_feature` jako parę opcjonalnych parametrów `feature_type` i `feature_coord`. Sprawdź, o ile szybciej działa teraz ekstraktor.

##### Zadanie 4b (0.5 pkt)

Zmodyfikuj swój klasyfikator tak, aby korzystał ze zredukowanego zbioru cech. Proponowane rozwiązania:
* (a) zmień parametr `features` w każdym z drzew, tak aby nie szukały cech w oryginalnym wektorze, a w zredukowanym,
* (b) nie zmieniaj klasyfikatora, a zamiast tego mapuj zredukowane cechy do wektora o oryginalnej długości.

Przetestuj cały pipeline na obrazach testowych.

---

#### Zadanie 5 (0.0 pkt | 1.0 pkt)

Wykonaj testy jakości detekcji dla klasyfikatorów o różnej liczbie cech. Wyniki omów, komentując mocne i słabe strony metody - ilościowe miary jakości nie są istotne (m.in. z tego powodu zbiór testowy jest tak niewielki i nie ma etykiet).

Wykonaj testy wydajnościowe różnych klasyfikatorów. Dla porównania uwzględnij przynajmniej jeden przed optymalizacją ekstrakcji cech. Wyniki przedstaw w formie tabeli, mierząc średni czas przetwarzania jednego obrazu.

#### Zadanie 6 (0.0 pkt | 0.5 pkt)

Wykorzystaj dowolny znaleziony w internecie model głębokiej sieci neuronowej do detekcji twarzy, np. [https://github.com/timesler/facenet-pytorch](https://github.com/timesler/facenet-pytorch), porównując jego działanie z uzyskanymi w trakcie laboratorium klasyfikatorami pod względem jakości detekcji i szybkości działania.